In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('./gdrive/MyDrive/AEGAN/中文_基于GAN的图像对抗样本防御2')
!nvidia-smi       
!ls

In [ ]:
!pip install adversarial-robustness-toolbox

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import DeepFool
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from art.defences.preprocessor import FeatureSqueezing
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import SpatialSmoothing
from art.defences.preprocessor import JpegCompression

In [4]:
class Generator(nn.Module):

    def __init__(self, in_ch):
        super(Generator, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, 64, 4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.deconv3 = nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.deconv4 = nn.ConvTranspose2d(64, in_ch, 4, stride=2, padding=1)

    def forward(self, x):
        h = F.leaky_relu(self.bn1(self.conv1(x)))
        h = F.leaky_relu(self.bn2(self.conv2(h)))
        h = F.leaky_relu(self.bn3(self.deconv3(h)))
        h = torch.tanh(self.deconv4(h))
        return h

In [ ]:
G1 = Generator(1).cuda()
G1.load_state_dict(torch.load("./checkpoint/test_mnist/1.tar")["generator"])
G1.eval()

G2 = Generator(1).cuda()
G2.load_state_dict(torch.load("./checkpoint/test_mnist/2.tar")["generator"])
G2.eval()

G3 = Generator(1).cuda()
G3.load_state_dict(torch.load("./checkpoint/test_mnist/3.tar")["generator"])
G3.eval()

G4 = Generator(1).cuda()
G4.load_state_dict(torch.load("./checkpoint/test_mnist/4.tar")["generator"])
G4.eval()

G5 = Generator(1).cuda()
G5.load_state_dict(torch.load("./checkpoint/test_mnist/5.tar")["generator"])
G5.eval()

G6 = Generator(1).cuda()
G6.load_state_dict(torch.load("./checkpoint/test_mnist/6.tar")["generator"])
G6.eval()

G7 = Generator(1).cuda()
G7.load_state_dict(torch.load("./checkpoint/test_mnist/7.tar")["generator"])
G7.eval()

G8 = Generator(1).cuda()
G8.load_state_dict(torch.load("./checkpoint/test_mnist/8.tar")["generator"])
G8.eval()

G9 = Generator(1).cuda()
G9.load_state_dict(torch.load("./checkpoint/test_mnist/9.tar")["generator"])
G9.eval()

G10 = Generator(1).cuda()
G10.load_state_dict(torch.load("./checkpoint/test_mnist/10.tar")["generator"])
G10.eval()

G11 = Generator(1).cuda()
G11.load_state_dict(torch.load("./checkpoint/test_mnist/11.tar")["generator"])
G11.eval()

G12 = Generator(1).cuda()
G12.load_state_dict(torch.load("./checkpoint/test_mnist/12.tar")["generator"])
G12.eval()

G13 = Generator(1).cuda()
G13.load_state_dict(torch.load("./checkpoint/test_mnist/13.tar")["generator"])
G13.eval()

G14 = Generator(1).cuda()
G14.load_state_dict(torch.load("./checkpoint/test_mnist/14.tar")["generator"])
G14.eval()

G15 = Generator(1).cuda()
G15.load_state_dict(torch.load("./checkpoint/test_mnist/15.tar")["generator"])
G15.eval()

G16 = Generator(1).cuda()
G16.load_state_dict(torch.load("./checkpoint/test_mnist/16.tar")["generator"])
G16.eval()

G17 = Generator(1).cuda()
G17.load_state_dict(torch.load("./checkpoint/test_mnist/17.tar")["generator"])
G17.eval()

In [ ]:
G18 = Generator(1).cuda()
G18.load_state_dict(torch.load("./apegan_mnis.tar")["generator"])
G18.eval()

In [7]:
test_loader = torch.utils.data.DataLoader(
            torchvision.datasets.MNIST(root = './', train=False, download=False,
                           transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()])),
            batch_size=128, shuffle=False)

In [8]:
class MnistCNN(nn.Module):

    def __init__(self):
        super(MnistCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc3 = nn.Linear(1024, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h = F.dropout2d(F.max_pool2d(h, 6), p=0.25)
        h = F.dropout2d(self.fc3(h.view(h.size(0), -1)), p=0.5)
        h = self.fc4(h)
        return F.log_softmax(h,dim=1)

In [ ]:
model = MnistCNN().cuda()
model.load_state_dict(torch.load("./CNN/mnist_cnn.tar")["state_dict"])
model.eval()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

In [ ]:
accuracy = 0
for x_test, y_test in test_loader:
  predictions = classifier.predict(x_test)
  accuracy += np.sum(np.argmax(predictions, axis=1) == y_test.numpy())
print('Accuracy on benign test examples:{}%'.format(accuracy/10000*100))

In [ ]:
# attack_fgsm = FastGradientMethod(estimator=classifier,eps=0.1)
# attack_fgsm = BasicIterativeMethod(estimator=classifier, eps=0.7)
# attack_fgsm = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma=0.8)
# attack_fgsm = DeepFool(classifier=classifier)
attack_fgsm = CarliniL2Method(classifier=classifier, confidence=0.01,binary_search_steps=15, max_iter=15)
adv_fgsm_accuracy = 0
re_accuracy = 0
num = 0
for x_test, y_test in test_loader:
  adv_fgsm = attack_fgsm.generate(x=x_test)
  # torch.save({"CW":adv_fgsm,"y":y_test}, "CW.tar")
  adv_fgsm_predictions = classifier.predict(adv_fgsm)
  adv_fgsm_accuracy += np.sum(np.argmax(adv_fgsm_predictions, axis=1) == y_test.numpy())

  re_x1 = G1(torch.from_numpy(adv_fgsm).cuda())
  re_y1 = classifier.predict(re_x1.cpu().detach().numpy())

  re_x2 = G2(torch.from_numpy(adv_fgsm).cuda())
  re_y2 = classifier.predict(re_x2.cpu().detach().numpy())

  re_x3 = G3(torch.from_numpy(adv_fgsm).cuda())
  re_y3 = classifier.predict(re_x3.cpu().detach().numpy())

  re_x4 = G4(torch.from_numpy(adv_fgsm).cuda())
  re_y4 = classifier.predict(re_x4.cpu().detach().numpy())

  re_x5 = G5(torch.from_numpy(adv_fgsm).cuda())
  re_y5 = classifier.predict(re_x5.cpu().detach().numpy())

  re_x6 = G6(torch.from_numpy(adv_fgsm).cuda())
  re_y6 = classifier.predict(re_x6.cpu().detach().numpy())

  re_x7 = G7(torch.from_numpy(adv_fgsm).cuda())
  re_y7 = classifier.predict(re_x7.cpu().detach().numpy())

  re_x8 = G8(torch.from_numpy(adv_fgsm).cuda())
  re_y8 = classifier.predict(re_x8.cpu().detach().numpy())

  re_x9 = G9(torch.from_numpy(adv_fgsm).cuda())
  re_y9 = classifier.predict(re_x9.cpu().detach().numpy())

  re_x10 = G10(torch.from_numpy(adv_fgsm).cuda())
  re_y10 = classifier.predict(re_x10.cpu().detach().numpy())

  re_x11 = G11(torch.from_numpy(adv_fgsm).cuda())
  re_y11 = classifier.predict(re_x11.cpu().detach().numpy())

  re_x12 = G12(torch.from_numpy(adv_fgsm).cuda())
  re_y12 = classifier.predict(re_x12.cpu().detach().numpy())

  re_x13 = G13(torch.from_numpy(adv_fgsm).cuda())
  re_y13 = classifier.predict(re_x13.cpu().detach().numpy())

  re_x14 = G4(torch.from_numpy(adv_fgsm).cuda())
  re_y14 = classifier.predict(re_x14.cpu().detach().numpy())

  re_x15 = G15(torch.from_numpy(adv_fgsm).cuda())
  re_y15 = classifier.predict(re_x15.cpu().detach().numpy())

  re_x16 = G16(torch.from_numpy(adv_fgsm).cuda())
  re_y16 = classifier.predict(re_x16.cpu().detach().numpy())

  re_x17 = G17(torch.from_numpy(adv_fgsm).cuda())
  re_y17 = classifier.predict(re_x17.cpu().detach().numpy())

  # re_y = re_y17
  # re_y = (re_y5 + re_y7 + re_y8 + re_y9 + re_y10 + re_y11 + re_y12 + re_y15 + re_y16 + re_y17)/10
  # re_y = (re_y5 + re_y7 + re_y8 + re_y9 + re_y10 + re_y11 + re_y12 + re_y15 + re_y16 + re_y17)/10
  # re_y = (re_y8 + re_y9 + re_y10 + re_y11 + re_y12 + re_y15 + re_y16 + re_y17)/8
  # re_y = (re_y5 + re_y7)/2
  # re_y = (re_y9 + re_y15 + re_y17)/3
  re_y = (re_y12 + re_y17)/2
  # re_y = (re_y8 + re_y9 + re_y10 + re_y11 + re_y12 + re_y15 + re_y16 + re_y17)/8
  # APEGAN
  # re_x = G18(torch.from_numpy(adv_fgsm).cuda())
  # re_y = classifier.predict(re_x.cpu().detach().numpy())
  # bit_depth
  # defense0 = FeatureSqueezing(clip_values=(0.0,0.8),bit_depth=1)
  # re_x = defense0(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)
  # TotalVarMin
  # defense2 = TotalVarMin()
  # re_x = defense2(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)
  # SpatialSmoothing
  # defense3 = SpatialSmoothing(window_size=2)
  # re_x = defense3(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)

  re_accuracy += np.sum(np.argmax(re_y, axis=1) == y_test.numpy())
  num = num + len(x_test)
  print(adv_fgsm_accuracy/num*100)
  print(re_accuracy/num*100)
  if num>100:
    break

In [ ]:
cw = torch.load('./CW.tar')
adv_fgsm = cw['CW']
y_test = cw['y']
adv_fgsm_predictions = classifier.predict(adv_fgsm)
adv_fgsm_accuracy = np.sum(np.argmax(adv_fgsm_predictions, axis=1) == y_test.numpy())

re_x1 = G1(torch.from_numpy(adv_fgsm).cuda())
re_y1 = classifier.predict(re_x1.cpu().detach().numpy())

re_x2 = G2(torch.from_numpy(adv_fgsm).cuda())
re_y2 = classifier.predict(re_x2.cpu().detach().numpy())

re_x3 = G3(torch.from_numpy(adv_fgsm).cuda())
re_y3 = classifier.predict(re_x3.cpu().detach().numpy())

re_x4 = G4(torch.from_numpy(adv_fgsm).cuda())
re_y4 = classifier.predict(re_x4.cpu().detach().numpy())

re_x5 = G5(torch.from_numpy(adv_fgsm).cuda())
re_y5 = classifier.predict(re_x5.cpu().detach().numpy())

re_x6 = G6(torch.from_numpy(adv_fgsm).cuda())
re_y6 = classifier.predict(re_x6.cpu().detach().numpy())

re_x7 = G7(torch.from_numpy(adv_fgsm).cuda())
re_y7 = classifier.predict(re_x7.cpu().detach().numpy())

re_x8 = G8(torch.from_numpy(adv_fgsm).cuda())
re_y8 = classifier.predict(re_x8.cpu().detach().numpy())

re_x9 = G9(torch.from_numpy(adv_fgsm).cuda())
re_y9 = classifier.predict(re_x9.cpu().detach().numpy())

re_x10 = G10(torch.from_numpy(adv_fgsm).cuda())
re_y10 = classifier.predict(re_x10.cpu().detach().numpy())

re_x11 = G11(torch.from_numpy(adv_fgsm).cuda())
re_y11 = classifier.predict(re_x11.cpu().detach().numpy())

re_x12 = G12(torch.from_numpy(adv_fgsm).cuda())
re_y12 = classifier.predict(re_x12.cpu().detach().numpy())

re_x13 = G13(torch.from_numpy(adv_fgsm).cuda())
re_y13 = classifier.predict(re_x13.cpu().detach().numpy())

re_x14 = G4(torch.from_numpy(adv_fgsm).cuda())
re_y14 = classifier.predict(re_x14.cpu().detach().numpy())

re_x15 = G15(torch.from_numpy(adv_fgsm).cuda())
re_y15 = classifier.predict(re_x15.cpu().detach().numpy())

re_x16 = G16(torch.from_numpy(adv_fgsm).cuda())
re_y16 = classifier.predict(re_x16.cpu().detach().numpy())

re_x17 = G17(torch.from_numpy(adv_fgsm).cuda())
re_y17 = classifier.predict(re_x17.cpu().detach().numpy())

re_y = re_y17

# bit_depth
# defense0 = FeatureSqueezing(clip_values=(0.0,0.8),bit_depth=1)
# re_x = defense0(adv_fgsm)[0]
# re_y = classifier.predict(re_x)
# TotalVarMin
# defense2 = TotalVarMin()
# re_x = defense2(adv_fgsm)[0]
# re_y = classifier.predict(re_x)
# SpatialSmoothing
# defense3 = SpatialSmoothing(window_size=2)
# re_x = defense3(adv_fgsm)[0]
# re_y = classifier.predict(re_x)


re_accuracy = np.sum(np.argmax(re_y, axis=1) == y_test.numpy())
print(adv_fgsm_accuracy/len(y_test)*100)
print(re_accuracy/len(y_test)*100)